In [ ]:
import os

import cv2
import onnx
from easydict import EasyDict

from onnx2keras import onnx_to_keras
from onnx_pytorch import code_gen


from adversarialbox.adversary import Adversary
from adversarialbox.attacks.gradient_method import FGSM
from advbox.models.keras import KerasModel
from tutorials.mnist_model_pytorch import Net

import tensorflow_datasets as tfds

from pytorchGenerator.model import Model
import numpy as np
import onnxruntime
import pandas as pd
import torchvision.models as models
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import eagerpy as ep

from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfPGD

import tensorflow as tf

# tf.compat.v1.disable_eager_execution()
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

from art.attacks.evasion import FastGradientMethod
from art.estimators.classification.keras import KerasClassifier
from art.estimators.classification import PyTorchClassifier
from art.utils import load_mnist

from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method


from matplotlib import pyplot as plt


In [ ]:
def sigmoid(input):
    return 1 / (1 + np.exp(-input))

In [ ]:
def onnx2Keras(path):
    onnx_model = onnx.load(path)
    model = onnx_to_keras(onnx_model, ['input_imgs', 'big_input_imgs', 'desire', 'traffic_convention', 'initial_state'],
                          name_policy='renumerate', change_ordering=False)
    model.save("models/keras" )

In [ ]:
def ld_mnist():
    """Load training and test data."""

    def convert_types(image, label):
        image = tf.cast(image, tf.float32)
        image /= 255
        return image, label

    dataset, info = tfds.load(
        "mnist",  with_info=True, as_supervised=True
    )
    mnist_train, mnist_test = dataset["train"], dataset["test"]
    mnist_train = mnist_train.map(convert_types).shuffle(10000).batch(128)
    mnist_test = mnist_test.map(convert_types).batch(128)
    return EasyDict(train=mnist_train, test=mnist_test)

In [ ]:
onnx_model = "models/supercombo.onnx"
kerasModel = tf.keras.models.load_model("models/keras", compile=False)
pytorchModel = torch.load('models/supercombo.pt')
pytorchModel.eval()

In [ ]:
(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_mnist()
data = ld_mnist()
model = Sequential()
model.add(Conv2D(filters=4, kernel_size=(5, 5), strides=1, activation="relu", input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=10, kernel_size=(5, 5), strides=1, activation="relu", input_shape=(23, 23, 4)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(100, activation="relu"))
model.add(Dense(10, activation="softmax"))

model.compile(loss=categorical_crossentropy, optimizer=Adam(learning_rate=0.01), metrics=["accuracy"])

In [ ]:
dataset = np.load('data/numpy.npz')

datasize = dataset['inputImgs'].shape[0]

inputImgs_data = dataset['inputImgs']
bigInputImgs_data = dataset['bigInputImgs']
desire_data = dataset['desire']
trafficConvention_data = dataset['trafficConvention']
initialState_data = dataset['initialState']
output_data = dataset['output']

Testing tool –Adversial Robustness 360 Toolbox–

In [ ]:
try:
    classifier = KerasClassifier(model=kerasModel, use_logits=False)
    attack = FastGradientMethod(estimator=classifier, eps=0.2)
    x_test_adv = attack.generate(x={"input_imgs": inputImgs_data,
                                    "big_input_imgs": bigInputImgs_data,
                                    "desire": desire_data,
                                    "traffic_convention": trafficConvention_data,
                                    "initial_state": initialState_data})
    predictions = classifier.predict(x_test_adv)
    accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
    print("Accuracy on adversarial test examples: {}%".format(accuracy * 100))
except Exception as e: print("Expected 1 input variable but got 5")

In [ ]:
try:
    classifier = KerasClassifier(model=model, clip_values=(min_pixel_value, max_pixel_value), use_logits=False)
    classifier.fit(x_train, y_train, batch_size=64, nb_epochs=3)
    predictions = classifier.predict(x_test)
    accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
    print("Clean Accuracy : {}%".format(accuracy * 100))
    attack = FastGradientMethod(estimator=classifier, eps=0.2)
    x_test_adv = attack.generate(x=x_test)
    predictions = classifier.predict(x_test_adv)
    accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
    print("Accuracy on adversarial test examples: {}%".format(accuracy * 100))
except Exception as e: print(e)

Testing tool –Cleverhans–

In [ ]:
try:
    test_acc_clean = tf.metrics.SparseCategoricalAccuracy()
    test_acc_fgsm = tf.metrics.SparseCategoricalAccuracy()
    for x, y in dataset:
        test_acc_clean(y, output_data)
        x_fgm = fast_gradient_method(kerasModel, x, 0.3, np.inf)
        y_pred_fgm = model(x_fgm)
        test_acc_fgsm(y, y_pred_fgm)

    print("test acc on clean examples (%): {:.3f}".format(test_acc_clean.result() * 100))
    print("test acc on FGM adversarial examples (%): {:.3f}".format(test_acc_fgsm.result() * 100))
except Exception as e: print("Expected 1 input variable but got 5")

In [ ]:
try:
    test_acc_clean = tf.metrics.SparseCategoricalAccuracy()
    test_acc_fgsm = tf.metrics.SparseCategoricalAccuracy()
    for x, y in data.test:
        y_pred = model(x)
        test_acc_clean(y, y_pred)

        x_fgm = fast_gradient_method(model, x, 0.3, np.inf)
        y_pred_fgm = model(x_fgm)
        test_acc_fgsm(y, y_pred_fgm)

    print("test acc on clean examples (%): {:.3f}".format(test_acc_clean.result() * 100))
    print("test acc on FGM adversarial examples (%): {:.3f}".format(test_acc_fgsm.result() * 100))
except Exception as e: print(e)

Testing tool –Foolbox–

In [ ]:
try:
    print("Foolbox worked")
except Exception as e: print("Expected 1 input variable but got 5")

In [ ]:

try:
    model = models.resnet18(pretrained=True).eval()
    preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
    fmodel = PyTorchModel(model, bounds=(0, 1), preprocessing=preprocessing)

    images, labels = ep.astensors(*samples(fmodel, dataset="imagenet", batchsize=16))
    clean_acc = accuracy(fmodel, images, labels)
    print(f"clean accuracy:  {clean_acc * 100:.1f} %")

    attack = LinfPGD()
    epsilons = [
        0.0,
        0.0002,
        0.0005,
        0.0008,
        0.001,
        0.0015,
        0.002,
        0.003,
        0.01,
        0.1,
        0.3,
        0.5,
        1.0,
    ]
    raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=epsilons)
    robust_accuracy = 1 - success.float32().mean(axis=-1)
    print("robust accuracy for perturbations with")
    for eps, acc in zip(epsilons, robust_accuracy):
        print(f"  Linf norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")
except Exception as e: print(e)

Testing tool –AdvBox–

In [ ]:
try:
    print("AdvBox worked")
except Exception as e: print(e)

In [ ]:
try:
    loss_func = torch.nn.CrossEntropyLoss()
    m = KerasModel(
        model, loss_func,(0, 1),
        channel_axis=1)
except Exception as e: print(e)

Testing tool –Foolbox–

In [ ]:
try:
    print("Foolbox worked")
except Exception as e: print("Expected 1 input variable but got 5")

In [ ]:

try:
    model = models.resnet18(pretrained=True).eval()
    preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
    fmodel = PyTorchModel(model, bounds=(0, 1), preprocessing=preprocessing)

    images, labels = ep.astensors(*samples(fmodel, dataset="imagenet", batchsize=16))
    clean_acc = accuracy(fmodel, images, labels)
    print(f"clean accuracy:  {clean_acc * 100:.1f} %")

    attack = LinfPGD()
    epsilons = [
        0.0,
        0.0002,
        0.0005,
        0.0008,
        0.001,
        0.0015,
        0.002,
        0.003,
        0.01,
        0.1,
        0.3,
        0.5,
        1.0,
    ]
    raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=epsilons)
    robust_accuracy = 1 - success.float32().mean(axis=-1)
    print("robust accuracy for perturbations with")
    for eps, acc in zip(epsilons, robust_accuracy):
        print(f"  Linf norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")
except Exception as e: print(e)

Testing tool –AdvBox–

In [ ]:
try:
    print("AdvBox worked")
except Exception as e: print(e)

In [ ]:
try:
    loss_func = torch.nn.CrossEntropyLoss()
    m = KerasModel(
        model, loss_func,(0, 1),
        channel_axis=1)
except Exception as e: print(e)